In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import random_pauli
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import BasisTranslator
from qiskit.circuit.library import UnitaryGate
import numpy as np
from typing import Optional
import random

In [4]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import random_statevector
from qiskit.circuit.library import XGate, YGate, ZGate
import random

def add_noise_to_circuit(p1: float, p2: float, circuit: QuantumCircuit) -> QuantumCircuit:
    """
    Add noise to a quantum circuit by inserting random Pauli operators after gates.
    
    Args:
        p1 (float): Probability of noise after single-qubit gates (0 to 1)
        p2 (float): Probability of noise after two-qubit gates (0 to 1)
        circuit (QuantumCircuit): Input quantum circuit
        
    Returns:
        QuantumCircuit: Circuit with added noise operations
    """
    if not (0 <= p1 <= 1 and 0 <= p2 <= 1):
        raise ValueError("Probabilities must be between 0 and 1")
    
    # Create a new circuit with the same registers
    noisy_circuit = QuantumCircuit(*circuit.qregs, *circuit.cregs)
    pauli_gates = [XGate(), YGate(), ZGate()]
    
    # Iterate through instructions in the original circuit
    for inst, qargs, cargs in circuit.data:
        # Add the original instruction
        noisy_circuit.append(inst, qargs, cargs)
        
        # Determine if we should add noise based on the gate type
        if len(qargs) == 1:  # Single-qubit gate
            if random.random() < p1:
                # Add random Pauli operator
                random_pauli = random.choice(pauli_gates)
                noisy_circuit.append(random_pauli, [qargs[0]])
                
        elif len(qargs) == 2:  # Two-qubit gate
            if random.random() < p2:
                # Add random Pauli operator to each qubit
                for qubit in qargs:
                    if random.random() < 0.5:  # 50% chance for each qubit
                        random_pauli = random.choice(pauli_gates)
                        noisy_circuit.append(random_pauli, [qubit])
    
    return noisy_circuit

In [5]:
p1, p2 = 0.1, 0.2

# Test the noise generator
def test_noise_generator():
    # Create a simple quantum circuit
    qr = QuantumRegister(2, 'q')
    cr = ClassicalRegister(2, 'c')
    circuit = QuantumCircuit(qr, cr)
    
    # Add some gates
    circuit.h(0)      # Hadamard on qubit 0
    circuit.cx(0, 1)  # CNOT between qubit 0 and 1
    circuit.h(1)      # Hadamard on qubit 1

    # Add noise with different probabilities

    noisy_circuit = add_noise_to_circuit(p1, p2, circuit)
    # noisy_circuit.measure(qr, cr)
    
    print("Original circuit depth:", circuit.depth())
    print("Noisy circuit depth:", noisy_circuit.depth())
    print("\nOriginal circuit:")
    print(circuit)
    print("\nNoisy circuit:")
    print(noisy_circuit)
    
    return circuit, noisy_circuit

# Run the test
original_circuit, noisy_circuit = test_noise_generator()

Original circuit depth: 3
Noisy circuit depth: 3

Original circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ H ├
          └───┘└───┘
c: 2/═══════════════
                    

Noisy circuit:
     ┌───┐          
q_0: ┤ H ├──■───────
     └───┘┌─┴─┐┌───┐
q_1: ─────┤ X ├┤ H ├
          └───┘└───┘
c: 2/═══════════════
                    


/var/folders/v4/gng44qs519q3hzsgrnlm91d40000gn/T/ipykernel_57557/716711159.py:27: DeprecationWarning: Treating CircuitInstruction as an iterable is deprecated legacy behavior since Qiskit 1.2, and will be removed in Qiskit 2.0. Instead, use the `operation`, `qubits` and `clbits` named attributes.
  for inst, qargs, cargs in circuit.data:


In [7]:
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary as sel
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.quantum_info import random_pauli
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import BasisTranslator
from qiskit.circuit.library import UnitaryGate
import numpy as np
from typing import Optional
import random

def convert_to_basis(circuit: QuantumCircuit) -> QuantumCircuit:

    # Define the target basis gates
    basis_gates = ['cx', 'id', 'rz', 'sx', 'x']
    
    # Create a PassManager with BasisTranslator using the session equivalence library
    pass_manager = PassManager()
    pass_manager.append(BasisTranslator(sel, target_basis=basis_gates))
    
    # Transform the circuit
    transformed_circuit = pass_manager.run(circuit)
    
    return transformed_circuit

qc = QuantumCircuit(3)
qc.h(0)
qc.z(1)
qc.x(0)
qc.measure_all()
print(convert_to_basis(qc))

global phase: 3π/4
        ┌─────────┐┌────┐┌─────────┐┌───┐ ░ ┌─┐      
   q_0: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├┤ X ├─░─┤M├──────
        └┬───────┬┘└────┘└─────────┘└───┘ ░ └╥┘┌─┐   
   q_1: ─┤ Rz(π) ├────────────────────────░──╫─┤M├───
         └───────┘                        ░  ║ └╥┘┌─┐
   q_2: ──────────────────────────────────░──╫──╫─┤M├
                                          ░  ║  ║ └╥┘
meas: 3/═════════════════════════════════════╩══╩══╩═
                                             0  1  2 


In [16]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_state_city
import numpy as np

def quantum_fourier_transform(circuit: QuantumCircuit, qubits: list, inverse: bool = False) -> None:
    n = len(qubits)
    
    if inverse:
        qubits = qubits[::-1]
    
    for i in range(n):
        circuit.h(qubits[i])
        for j in range(i+1, n):
            phase = 2 * np.pi / (2**(j-i+1))
            if inverse:
                phase = -phase
            circuit.cp(phase, qubits[j], qubits[i])
    
    if not inverse:
        for i in range(n//2):
            circuit.swap(qubits[i], qubits[n-i-1])

# Test the QFT
def test_qft():
    # Create a circuit with 3 qubits
    n_qubits = 3
    qc = QuantumCircuit(n_qubits, n_qubits)  # Include classical bits for measurement
    
    # Prepare an interesting initial state
    qc.h(0)    # Put qubit 0 in superposition
    qc.x(1)    # Put qubit 1 in |1⟩ state
    
    print("Circuit before QFT:")
    print(qc)
    
    # Apply QFT
    quantum_fourier_transform(qc, list(range(n_qubits)))
    
    print("\nCircuit after QFT:")
    print(qc)
    
    # Add measurements
    qc.measure_all()
    
    # Simulate the circuit
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit, shots=1000).result()
    
    # Get counts
    counts = result.get_counts()
    print("\nMeasurement counts:", counts)

# Run the test
test_qft()

Circuit before QFT:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
c: 3/═════
          

Circuit after QFT:
     ┌───┐┌───┐                                        
q_0: ┤ H ├┤ H ├─■────────■───────────────────────────X─
     ├───┤└───┘ │P(π/2)  │       ┌───┐               │ 
q_1: ┤ X ├──────■────────┼───────┤ H ├─■─────────────┼─
     └───┘               │P(π/4) └───┘ │P(π/2) ┌───┐ │ 
q_2: ────────────────────■─────────────■───────┤ H ├─X─
                                               └───┘   
c: 3/══════════════════════════════════════════════════
                                                       

Measurement counts: {'000 000': 259, '010 000': 253, '001 000': 236, '011 000': 252}


In [ ]:
def quantum_sum(a: int, b: int, n_qubits: Optional[int] = None) -> QuantumCircuit:
    """
    Implement quantum addition using the Draper adder algorithm.
    """
    if n_qubits is None:
        n_qubits = max(a.bit_length(), b.bit_length()) + 1
    
    qr = QuantumRegister(2 * n_qubits)
    cr = ClassicalRegister(n_qubits)
    circuit = QuantumCircuit(qr, cr)
    
    for i in range(n_qubits):
        if (a >> i) & 1:
            circuit.x(qr[i])
        if (b >> i) & 1:
            circuit.x(qr[i + n_qubits])
    
    quantum_fourier_transform(circuit, list(range(n_qubits)))
    
    for i in range(n_qubits):
        for j in range(n_qubits):
            phase = 2 * np.pi * (2**j) / (2**n_qubits)
            circuit.cp(phase, qr[i + n_qubits], qr[j])
    
    quantum_fourier_transform(circuit, list(range(n_qubits)), inverse=True)
    
    circuit.measure(qr[:n_qubits], cr)
    
    return circuit